In [ ]:
# Importando as bibliotecas

In [382]:
import requests as r
import pandas as pd
from datetime import datetime
import numpy as np
from pandasql import sqldf
import pyodbc as odc


# Requisitando os dados na API

In [381]:
url = 'https://covid19-brazil-api.now.sh/api/report/v1'
df = r.request('GET',url)

# Convertendo o Json em um DataFrame

In [411]:
#Criando um DataFrame vazio.
df2 = pd.DataFrame()
i = 0
#Obtendo a data atual do sistema.
data_atual = datetime.today().strftime('%Y-%m-%d')

#Lendo linha a linha do dicionaro 'Data' vindo do arquivo Json.
#O dicionario e composto por chave e valor.

for i in range(len(df.json()['data'])):
    df1 = pd.DataFrame(df.json()['data'][i], index=[0])
    
    #Alterando o formato da data
    df1['datetime'] = pd.to_datetime(df1['datetime']).dt.strftime("%Y-%m-%d")
    
    #Adicionando os registros ao dataframe = df2
    df2 = df2.append(df1)


# Fazer Select no Banco para conferir valores repetidos.

In [273]:
def SelectBanco ():
    
    con_string = ('DRIVER={SQL Server};'
              'SERVER=DESKTOP-AF7BA3M;'
              'DATABASE=CovidDB;')

    cnxn = odc.connect(con_string)
    query = """SELECT * FROM Tb_Covid1"""
    #Passando o retorno da consulta para um dataframe = 'result'
    result = pd.read_sql(query, cnxn)
    return result

# Verificando a data do ultimo registro no Banco para não inserir valores repetidos.

In [406]:
def IncrementoBanco (result,data_atual,df2):
    df0 = pd.DataFrame(result)
    if df0['datetime'].max() < data_atual :
            GravarNoBanco(df2)   

#  Monstando o Insert - SQL

In [265]:
def GravarNoBanco(df):

        cnxn = odc.connect('DRIVER={SQL Server};'
                       'SERVER=DESKTOP-AF7BA3M;'
                        'DATABASE=CovidDB;')
        cursor = cnxn.cursor()

        for index , linha in df.iterrows():
            cursor.execute("insert into Tb_Covid1(uid, uf, state,cases,deaths,suspects,refuses,datetime) values (?,?,?,?,?,?,?,?)",linha.uid,linha.uf,linha.state,linha.cases,linha.deaths,linha.suspects,linha.refuses,linha.datetime)
            cursor.commit()

# Instancioando a função de incremento no banco de dados.

In [412]:
IncrementoBanco(result,data_atual,df2)

# Consultando o banco para verificar os valores inseridos.

In [420]:
query = SelectBanco()
query

,id,uid,uf,state,cases,deaths,suspects,refuses,datetime
0,1001,35,SP,São Paulo,5337459,167847,5334,596,2022-04-16T22:34:57.620Z
1,1002,31,MG,Minas Gerais,3349540,61113,925,104,2022-04-16T22:34:57.620Z
2,1003,41,PR,Paraná,2434237,43035,400,119,2022-04-16T22:34:57.620Z
3,1004,43,RS,Rio Grande do Sul,2308860,39205,416,330,2022-04-16T22:34:57.620Z
4,1005,33,RJ,Rio de Janeiro,2117270,73128,1254,148,2022-04-16T22:34:57.620Z
...,...,...,...,...,...,...,...,...,...
75,1156,17,TO,Tocantins,304142,4148,13,0,2022-04-19
76,1157,27,AL,Alagoas,297279,6919,43,22,2022-04-19
77,1158,16,AP,Amapá,160370,2129,15,0,2022-04-19
78,1159,14,RR,Roraima,155385,2147,9,0,2022-04-19


# Agrupando e somando os valores por estado.

In [421]:
query[['uid','uf','cases','deaths','suspects','refuses']].groupby(['uid','uf']).sum().reset_index()

,uid,uf,cases,deaths,suspects,refuses
0,11,RO,1195948,21582,183,6
1,12,AC,248917,3992,24,0
2,13,AM,1745522,42504,54,78
3,14,RR,466135,6441,27,0
4,15,PA,2282725,54577,129,69
5,16,AP,481109,6385,45,0
6,17,TO,912112,12444,39,0
7,21,MA,1288284,32634,375,39
8,22,PI,1103983,23205,36,6
9,23,CE,3730431,80551,1479,267
